In [1]:
# Import required libraries
import numpy as np
import math
import datetime as dt
import pandas as pd
import pathlib

In [2]:
# Load data
# get relative data folder
PATH = pathlib.Path('./data').parent
DATA_PATH = PATH.joinpath("data").resolve()
PIDAR = pd.read_csv(DATA_PATH.joinpath("pidar.csv"))
AT = pd.read_csv(DATA_PATH.joinpath("DS_Asistencia_Tecnica.csv"))
EVA = pd.read_csv(DATA_PATH.joinpath('EVA_cultivos_modificado.csv'))
dfs = [PIDAR,AT,EVA]

In [3]:
EVA.columns

Index(['COD_DEP', 'DEPARTAMENTO', 'COD_MUN', 'MUNICIPIO', 'GRUP_CULTIVO',
       'SUBGRUP_CULTIVO', 'CULTIVO', 'DESAGREGACION_SIST_PRODUCTIVO', 'YEAR',
       'PERIODO', 'AREA_SEMBR', 'AREA_COSECH', 'PRODUCCION', 'RENDIMIENTO',
       'ESTADO_PRODUCCION', 'NOMBRE_CIENTI', 'CICLO_CULTIVO',
       'CADENA_PRODUCTIVA_ADR', 'NIVEL', 'ID_ADR'],
      dtype='object')

In [6]:
# #Agrupar por departamtento cada una de las columnas
dff = EVA.groupby(['DEPARTAMENTO','YEAR','CADENA_PRODUCTIVA_ADR']).sum().reset_index()
dff

,DEPARTAMENTO,YEAR,CADENA_PRODUCTIVA_ADR,COD_DEP,COD_MUN,AREA_SEMBR,AREA_COSECH,PRODUCCION,RENDIMIENTO,NIVEL,ID_ADR
0,AMAZONAS,2006,ARROZ,273,274339,11,11,16,4.35,9,37527
1,AMAZONAS,2006,MAIZ,455,457151,204,203,468,7.86,15,62520
2,AMAZONAS,2007,ACHICORIA,182,182541,6,6,6,2.00,6,21610
3,AMAZONAS,2007,AHUYAMA,91,91540,10,10,38,3.80,3,11213
4,AMAZONAS,2007,AJI,182,182541,28,28,35,2.50,6,22418
...,...,...,...,...,...,...,...,...,...,...,...
16340,VICHADA,2018,PALMA DE ACEITE,297,298921,8409,7100,28558,8.34,9,37818
16341,VICHADA,2018,PIÑA,198,198774,63,51,355,12.00,6,23002
16342,VICHADA,2018,PLATANO,396,397922,2111,511,4126,28.00,12,45636
16343,VICHADA,2018,SACHA INCHI,198,199297,11,11,21,4.00,6,25214
